In [5]:
import pdfplumber
import openai
from fpdf import FPDF

class PDF(FPDF):
    def add_section(self, title, body):
        self.add_page()
        self.set_font('Arial', 'B', 12)
        self.cell(0, 10, title, 0, 1, 'L')
        self.set_font('Arial', '', 12)
        self.multi_cell(0, 10, body)
        self.ln(10)

def generate_advanced_prompt(title, subsection, content):
    descriptions = {
        "Preferred Provision": "Translate the legal language into plain English while preserving the original intent and accuracy.",
        "Drafter’s Note": "Rewrite the drafter’s technical notes into straightforward guidance that can be understood by non-experts.",
        "Negotiation Commentary": "Condense the negotiation commentary into clear, actionable advice for business stakeholders.",
        "Description of Objection or Problem": "Clarify the legal objections or problems raised, ensuring that the essence of the issues is communicated clearly to a non-legal audience.",
        "Fallback Provision": "Simplify the fallback provisions to make them accessible and understandable, while ensuring that all legal bases are covered.",
        "Red Flag Provision": "Explain the critical concerns outlined in the red flag provisions in simple terms, highlighting potential legal risks and implications."
    }

    prompt = (
        f"Context: This text is extracted from a legal playbook titled '{title}', specifically from the subsection '{subsection}'. "
        f"The text outlines complex legal guidelines and requirements which are essential for legal compliance and negotiations. "
        f"Task: Please simplify the following content into clear, straightforward language suitable for readers without a legal background. "
        f"Ensure all legal concepts are accurately translated and easy to understand.\n\n"
        f"{descriptions.get(subsection, 'Please simplify this content')}:\n'{content}'"
    )
    return prompt

def simplify_text(prompt):
    response = openai.ChatCompletion.create(
        engine="gpt-4",
        prompt=prompt,
        max_tokens=8192,
        temperature=0.7
    )
    return response.choices[0].text.strip()

def extract_and_process_sections(pdf_path, output_pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        pdf_output = PDF()
        for page in pdf.pages:
            text = page.extract_text()
            if text:
                lines = text.split('\n')
                for idx, line in enumerate(lines):
                    if line.startswith("Issue"):
                        title = line.strip()
                        section_text = '\n'.join(lines[idx+1:])  # get all text until the next section
                        break_pos = section_text.find("Issue", 1)  # find the start of the next section
                        if break_pos != -1:
                            section_text = section_text[:break_pos]  # trim text to the current section only

                        prompt = generate_advanced_prompt(title, "Full Section", section_text)
                        simplified_text = simplify_text(prompt)
                        pdf_output.add_section(title, simplified_text)
                        break  # process one section at a time
        pdf_output.output(output_pdf_path)
        print("PDF generated successfully.")

# Setup your OpenAI API key
openai.api_key = 'api-key'

# Specify the input and output PDF paths
input_pdf_path = 'playbook.pdf'
output_pdf_path = 'simplified_playbook3.pdf'
extract_and_process_sections(input_pdf_path, output_pdf_path)


InvalidRequestError: Invalid URL (POST /v1/engines/gpt-3.5-turbo/chat/completions)